### 最近接到任務，必須好好練習一下Python，又剛好想要查農林（2913）的歷史紀錄，所以順手做了一個可以快速查詢的小工具。預想的情境如下：

#### 1. 利用terminal查詢。
#### 2. 可以跑出互動式K線圖。
#### 3. 要有月線

## 開工！

## Import Modules

In [1]:
# -*- coding: utf-8 -*-
import sys
from plotly.tools import FigureFactory as FF
import plotly
from pandas_datareader import data
from datetime import datetime
from plotly.graph_objs import Scatter, Layout, Line
import numpy as np

### 1. 要注意第一行，特別註明encoding，否則只要出現中文，就會報錯。
### 2. `sys`是用來讓我可以直接在terminal把參數放進去。
### 3. 過去用`pandas.io.data`來抓資料，不過這Module將被移除了，改為pandas_datareader，只是用法些微不同。

## 查詢＋畫圖

In [2]:
# 算均線
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

# Query and Draw
def query_stock(stock, start='2011.01.01', end='today'):
    stock+=".TW"
    # datetime的用法超好用，這裡將字串轉成時間格式
    st = datetime.date(datetime.strptime(start, '%Y.%m.%d'))

    if end == 'today':
        en = datetime.date(datetime.today())
    else:
        en = datetime.date(datetime.strptime(end, '%Y.%m.%d'))
    # 新的抓股價的function直接幫Yahoo! Finance寫一個function
    df = data.get_data_yahoo(stock, st, en)
    fig = FF.create_candlestick(df.Open, df.High, df.Low, df.Close, dates=df.index)
    SMA20=moving_average(df.Close.values, 20)
    add_line = Scatter(
        x=df.index[-len(SMA20):],
        y=SMA20,
        name= '20SMA',
        line=Line(color='rgb(200, 200, 250)')
        )
    fig['data'].extend([add_line])
    plotly.offline.plot(fig, validate=False)

## 從terminal抓參數
###這邊利用`sys.argv`將參數放入function，`sys.argv`是一個List會抓取terminal上放在python後面的參數，List裡的第一個元素會是所執行的.py檔，後面通常會放參數。

In [4]:
if len(sys.argv) < 2:
    print 'no argument'
    sys.exit()
elif len(sys.argv) == 2:
    query_stock(sys.argv[1])
elif len(sys.argv) == 3:
    query_stock(sys.argv[1],sys.argv[2])
elif len(sys.argv) == 4:
    query_stock(sys.argv[1],sys.argv[2],sys.argv[3])

## 在terminal執行

In [ ]:
python query_stock.py 1215 2015.01.01 2015.06.30

### 補充
#### 原本是想要固定每次都抓過去五年的紀錄，可以利用`timedelta`來執行，今天扣掉五年的天數，即為開始的日期，用法：`datetime.date(datetime.today())-timedelta(days=365*5)`，但後來認為應該還是要保留一些彈性，所以作罷。